In [ ]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import requests
options = Options()
options.add_argument('--incognito')
driver = webdriver.Chrome(options=options)
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'
driver.get(url)
nuorodos = driver.find_elements(By.CLASS_NAME, 'ipc-title-link-wrapper')
for nuoroda in nuorodos:
    href = nuoroda.get_attribute('href')
    with open('filmu_nuorodos.txt', 'a+') as failas:
        failas.write(f'{href}\n')

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

# Nustatome Chrome parinktis
options = Options()
options.add_argument('--incognito')

# Inicijuojame Chrome WebDriver
driver = webdriver.Chrome(options=options)

# Skaitome nuorodas iš failo
with open('filmu_nuorodos.txt', 'r') as failas:
    nuorodos = failas.readlines()

filmu_sarasas = []

try:
    for idx, nuoroda in enumerate(nuorodos):
        filmas = {}
        nuoroda = nuoroda.strip() 
        driver.get(nuoroda)


        try:
            pavadinimas_element = driver.find_elements(By.CSS_SELECTOR, 'div h1[class="sc-d8941411-0"]')
            pavadinimas = [pavadinimas.text.strip() for pavadinimas in pavadinimas_element]
            # print(pavadinimas)
        except NoSuchElementException:
            pavadinimas = 'N/A'
        filmas['pavadinimas'] = pavadinimas


        # Ištraukiame aktorių vardus
        try:
            aktoriai = driver.find_elements(By.CSS_SELECTOR, 'a[href*="/name/"][data-testid="title-cast-item__actor"]')
            aktoriai_text = [aktorius.text for aktorius in aktoriai]
        except NoSuchElementException:
            aktoriai_text = ['N/A']
        filmas['aktoriai'] = aktoriai_text


        try:
            rezisierius = driver.find_element(By.CSS_SELECTOR, 'a[class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]')
            rezisierius = rezisierius.text.strip()
        except NoSuchElementException:
            rezisierius = ['N/A']
        filmas['rezisierius'] = rezisierius

        # Ištraukiame išleidimo metus
        try:
            metai_element = driver.find_element(By.CSS_SELECTOR, 'li[data-testid="title-details-releasedate"] div ul li a')
            metai = metai_element.text.strip()
        except NoSuchElementException:
            metai = 'N/A'
        filmas['metai'] = metai


        # Ištraukiame žanrus
        try:
            zanrai_elements = driver.find_elements(By.CSS_SELECTOR, 'div[data-testid="genres"] a')
            zanrai = [zanras.text.strip() for zanras in zanrai_elements]
        except NoSuchElementException:
            zanrai = ['N/A']
        filmas['zanrai'] = zanrai


        # Ištraukiame sertifikatą (Certificate)
        try:
            sertifikatas_element = driver.find_elements(By.CSS_SELECTOR, 'a[class="ipc-link ipc-link--baseAlt ipc-link--inherit-color"]')
            sertifikatas = [sertifikatas.text.strip() for sertifikatas in sertifikatas_element]
        except NoSuchElementException:
            sertifikatas = 'N/A'
        try:
            filmas['sertifikatas'] = sertifikatas[6]
        except Exception as e:
            filmas['sertifikatas'] = 'N/A'
        

        # Ištraukiame biudžetą (Budget)
        try:
            biudzetas_element = driver.find_elements(By.CSS_SELECTOR, 'span[class="ipc-metadata-list-item__list-content-item"]')
            biudzetas = [biudzetas.text.strip() for biudzetas in biudzetas_element]
        except NoSuchElementException:
            biudzetas = 'N/A'
        try:
            filmas['biudzetas'] = biudzetas[2]
        except Exception as e:
            filmas['biudzetas'] = 0


        # Ištraukiame pasaulines pajamas (Gross worldwide)
        try:
            pajamos_element = driver.find_elements(By.CSS_SELECTOR, 'span[class="ipc-metadata-list-item__list-content-item"]')
            pajamos = [pajamos.text.strip() for pajamos in pajamos_element]
        except NoSuchElementException:
            pajamos = 'N/A'
        try:
            filmas['pajamos'] = pajamos[6]
        except Exception as e:
            filmas['pajamos'] = 0


        # Ištraukiame filmo kalbą
        try:
            kalba_element = driver.find_elements(By.CSS_SELECTOR, 'a[class="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link"]')
            kalba = [kalba.text.strip() for kalba in kalba_element]
        except NoSuchElementException:
            kalba = 'N/A'
        try:
            filmas['kalba'] = kalba[19]
        except Exception as e:
            filmas['kalba'] = 'unknown'


        # Ištraukiame filmo trukmę
        try:
            trukme_element = driver.find_elements(By.CSS_SELECTOR, 'div[class="ipc-metadata-list-item__content-container"]')
            trukme =[trukme.text.strip() for trukme in trukme_element]
        except NoSuchElementException:
            trukme = 'N/A'
        try:
            filmas['trukme'] = trukme[-3]
        except Exception as e:
            filmas['trukme'] = 0


        # Ištraukiame IMDb reitingą
        try:
            imdb_reitingas_element = driver.find_element(By.CSS_SELECTOR, 'span[class="sc-bde20123-1 cMEQkK"]')
            imdb_reitingas = imdb_reitingas_element.text.strip()
        except NoSuchElementException:
            imdb_reitingas = 'N/A'
        filmas['imdb_reitingas'] = imdb_reitingas

        filmu_sarasas.append(filmas)
        

finally:
    driver.quit()

df = pd.DataFrame(filmu_sarasas)
df.to_csv('movies_top_250.csv', index=False)